In [1]:
import numpy as np

def nadaraya_watson_estimator(X_train, Y_train, X_test, h):
    """
    Nadaraya-Watson Kernel Regression Estimator (using Gaussian Kernel)
    Estimates E[Y|X=x] for points in X_test given training data (X_train, Y_train).

    Args:
        X_train (np.array): Explanatory variables (shape (M, d) or (M,))
        Y_train (np.array): Dependent variables (shape (M,) or (M, 1))
        X_test (np.array): Points where to estimate the conditional expectation (shape (M, d) or (M,))
        h (float): Bandwidth parameter (smoothing parameter)

    Returns:
        np.array: Estimated conditional expectations for X_test (shape (M,) or (M, 1))
    """
    
    # Ensure inputs are 2D arrays for dimension consistency
    if X_train.ndim == 1: X_train = X_train[:, None]
    if Y_train.ndim == 1: Y_train = Y_train[:, None]
    if X_test.ndim == 1: X_test = X_test[:, None]
        
    M_train, d = X_train.shape
    M_test, _ = X_test.shape
    
    # Calculate squared Euclidean distances between test points and training points
    # distances2[i, j] = ||X_test[i] - X_train[j]||^2
    # Use broadcasting for efficiency
    # Note: this might be memory-intensive for very large M and d
    distances2 = np.sum((X_test[:, None, :] - X_train[None, :, :])**2, axis=2)
    
    # Gaussian Kernel weights calculation
    # K_h(u) = (1 / (h*sqrt(2pi)))^d * exp(-0.5 * (u/h)^2)
    # Weights for each test point across all training points
    weights = np.exp(-0.5 * distances2 / (h**2))
    
    # Normalize weights so they sum to 1 for each test point (row-wise sum)
    # The normalization constant (1 / (h*sqrt(2pi)))^d cancels out in the ratio
    weights_normalized = weights / np.sum(weights, axis=1, keepdims=True)
    
    # Calculate the weighted average: E[Y|X=x] = sum(W_i(x) * Y_i)
    # Resulting shape (M_test, 1)
    Y_pred = np.sum(weights_normalized[:, :, None] * Y_train[None, :, :], axis=1)
    
    # Return 1D array if original Y was 1D
    return Y_pred.flatten() if Y_train.shape[1] == 1 else Y_pred



In [ ]:
# Problem setup
T = 1.0          # Terminal time
N = 10          # Number of time steps
dt = T / N       # Time step size
d = 1            # Dimension of Brownian motion
M = 10000    

In [13]:
def sigma(t, x):
    return np.ones((M, d))  # Example: constant volatility

def sigma_inv(t, x):
    return np.ones((M, d))  # Inverse of sigma (identity for constant sigma)

def H(t, x, z):
    return -0.5 * np.sum(z**2, axis=1)  # Example Hamiltonian

def g(x):
    return np.sum(x**2-x, axis=1)  # Terminal condition


In [14]:
X = np.zeros((N + 1, M, d))
Y = np.zeros((N + 1, M))
Z = np.zeros((N, M, d))
dW = np.random.normal(0, np.sqrt(dt), size=(N, M, d))

In [15]:
for n in range(N):
    X[n + 1] = X[n] + sigma(n * dt, X[n]) * dW[n]

In [16]:
# ... (Previous code for setup and forward simulation of X) ...

# Backward recursion for BSDE using Kernel Regression
Y[N] = g(X[N]).flatten() if X[N].ndim > 1 else g(X[N]) # Ensure Y[N] is 1D

# Define a simple constant bandwidth (needs tuning in practice)
# A typical rule of thumb might be h = np.std(X[n]) * M**(-1/(d+4))
h_bandwidth = 0.5 # Example value, adjust as needed

print("Performing backward recursion for BSDE using Kernel Regression...")

for n in reversed(range(N)):
    t_n = n * dt
    
    # --- Estimate conditional expectations using Kernel Regression ---
    
    # E[Y_{n+1} | X_n]: 
    # X_train = X[n], Y_train = Y[n+1], X_test = X[n]
    EY = nadaraya_watson_estimator(X[n], Y[n+1], X[n], h_bandwidth)
    
    # E[Y_{n+1} * dW_n | X_n]:
    # We need to estimate the conditional expectation for each dimension of dW
    # YdW is of shape (M, d)
    YdW = Y[n+1][:, None] * dW[n] 
    EYdW = np.zeros((M, d))
    for dim in range(d):
        EYdW[:, dim] = nadaraya_watson_estimator(X[n], YdW[:, dim], X[n], h_bandwidth)

    # --- Compute Z and Y using the BSDE discretization formula ---
    
    Z[n] = (1 / dt) * sigma_inv(t_n, X[n]) * EYdW
    Y[n] = EY + H(t_n, X[n], Z[n]) * dt

print("Backward recursion complete.")
print("-" * 30)

# --- Output and Visualization (as before) ---
approx_Y0 = np.mean(Y[0])
print(f"Approximate Y_0: {approx_Y0}")
theoretical_Y0 = T
print(f"Theoretical Y_0 (for this specific problem): {theoretical_Y0}")


Performing backward recursion for BSDE using Kernel Regression...
Backward recursion complete.
------------------------------
Approximate Y_0: 0.5167329919847594
Theoretical Y_0 (for this specific problem): 1.0
